# Random Effects Estimation of the determinants of leverage amongst SGX listed companies

## Import the necessary packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

import statsmodels.api as sm
from statsmodels.regression.linear_model import RegressionResults
from linearmodels.panel import RandomEffects
from linearmodels.panel.results import PanelResults, RandomEffectsResults, PanelEffectsResults

from statsmodels.iolib import load_pickle, save_pickle

from typing import Union

## Loading the SGX Data

In [ ]:
sgx = pd.read_csv("data/clean_sgx.csv")
sgx

In [ ]:
sgx = sgx.set_index(['Company Code', 'Year'], drop= False)

sgx

## Random Effects Model

### 1-Way Random Effects Estimation

#### 1-Way Entity Random Effects Model

In [ ]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = sgx[endo_var]
exog = sm.add_constant(sgx[exog_vars])

entity_re_mod = RandomEffects(endo, exog)

entity_re_fit = entity_re_mod.fit()

print(entity_re_fit.summary)

#### 1-Way Time Random Effects Model

In [ ]:
sgx = sgx.swaplevel('Year', 'Company Code')

endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = sgx[endo_var]
exog = sm.add_constant(sgx[exog_vars])

time_re_mod = RandomEffects(endo, exog)

time_re_fit = time_re_mod.fit()

print(time_re_fit.summary)

### 2-Way Random Effects Model

### Feasible Generalised Least Square Estimator

In [ ]:
def get_weighting_matrix(time_panels: pd.Series, entity_panels: pd.Series):
    t = time_panels.nunique()
    n = entity_panels.nunique()
    
    J_n_bar = (1 / n) * np.ones(shape = (n, n))
    J_t_bar = (1 / t) * np.ones(shape = (t, t))
    I_n = np.identity(n = n)
    I_t = np.identity(n = t)

    E_n = I_n - J_n_bar
    E_t = I_t - J_t_bar

    Q_1 = np.kron(E_n, E_t)
    Q_2 = np.kron(E_n, J_t_bar)
    Q_3 = np.kron(J_n_bar, E_t)
    Q_4 = np.kron(J_n_bar, J_t_bar)
    
    return np.array([Q_1, Q_2, Q_3, Q_4])

def get_omega_i(weighting_matrix: np.array, resid: np.array):

    w_1 = (resid.T @ weighting_matrix[0] @ resid) / np.trace(weighting_matrix[0])
    w_2 = (resid.T @ weighting_matrix[1] @ resid) / np.trace(weighting_matrix[1])
    w_3 = (resid.T @ weighting_matrix[2] @ resid) / np.trace(weighting_matrix[2])
    w_4 = w_2 + w_3 - w_1

    return np.array([w_1, w_2, w_3, w_4])

def get_rcorr_matrix(omega_matrix: np.array, weighting_matrix: np.array):
    omega = omega_matrix
    weight = weighting_matrix
    return omega[0] * weight[0] + omega[1] * weight[1] + omega[2] * weight[2] + omega[3] * weight[3]

def TwoWayRandomEffects(Y: pd.Series, X: Union[pd.Series, pd.DataFrame], entity_panel: pd.Series, time_panel: pd.Series, epsilon: float= 0.0001, maxiter: int= 99):
    # Step 1: Run OLS of Y on X
    ols = sm.OLS(Y, X)
    residuals = ols.fit().resid
    # Step 2: Get OLS weighting matrix
    weight_matrix = get_weighting_matrix(time_panels= time_panel, entity_panels= entity_panel)
    omega_matrix = get_omega_i(weight_matrix, residuals)
    OMEGA = get_rcorr_matrix(omega_matrix, weight_matrix)
    
    # Step 3: Get GLS residuals using weighting matrix
    gls = sm.GLS(endog= Y, exog= X, sigma= OMEGA)
    gls_residuals = gls.fit().resid
    # Step 4: Update GLS weighting matrix
    weight_matrix = get_weighting_matrix(time_panels= time_panel, entity_panels= entity_panel)
    omega_matrix = get_omega_i(weight_matrix, gls_residuals)
    OMEGA = get_rcorr_matrix(omega_matrix, weight_matrix)
    # Step 5: Update GLS coefficient estimates
    init_gls = ols ## Initial GLS model
    iter_gls = sm.GLS(endog= Y, exog= X, sigma= OMEGA) ## Updated GLS model

    i = 1
    while np.max(abs(init_gls.fit().params - iter_gls.fit().params)) >= epsilon: ## If there is a significant difference in the model estimates, re-run the refining steps
        init_gls = iter_gls ## Set the initial GLS model to the most updated model
        # Step 3: Get GLS residuals using weighting matrix
        gls = sm.GLS(endog= Y, exog= X, sigma= OMEGA)
        gls_residuals = gls.fit().resid
        # Step 4: Update GLS weighting matrix
        weight_matrix = get_weighting_matrix(time_panels= time_panel, entity_panels= entity_panel)
        omega_matrix = get_omega_i(weight_matrix, gls_residuals)
        OMEGA = get_rcorr_matrix(omega_matrix, weight_matrix)
        # Step 5: Update GLS coefficient estimates
        iter_gls = sm.GLS(endog= Y, exog= X, sigma= OMEGA) ## Produce an updated GLS model
        i += 1
        if i == maxiter:
            print(f"Maximum of {maxiter} iterations reached before model convergence was achieved.")
            break

    print(f"{i} iterations of GLS re-specification performed")
    return gls

In [ ]:
sgx = sgx.swaplevel('Company Code', 'Year')

endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = sgx[endo_var]
exog = sm.add_constant(sgx[exog_vars])

entity_panel = sgx['Company Code']
time_panel = sgx['Year']

tw_re_mod = TwoWayRandomEffects(endo, exog, entity_panel, time_panel)

tw_re_fit = tw_re_mod.fit()

print(tw_re_fit.summary())

### Test for significance of Random Effects

#### Lagrange Multiplier Test

In [ ]:
def cond_LM_stat(restricted_model: PanelResults, how: str): 
    T = restricted_model.time_info.total.astype('int')
    n = restricted_model.entity_info.total.astype('int')

    J_n = np.ones(shape = (n, n))
    J_T = np.ones(shape = (T, T))

    I_n = np.identity(n = n)
    I_T = np.identity(n = T)

    u_tilda = restricted_model.resids
    
    if how.lower() == 'entity':
        LM_C_entity = ((n * T) / (2 * (T - 1))) * (1 - ((u_tilda.T @ np.kron(I_n, J_T) @ u_tilda) / (u_tilda.T @ u_tilda))) ** 2
        return LM_C_entity
    elif how.lower() == 'time':
        LM_C_time = ((n * T) / (2 * (n - 1))) * (1 - ((u_tilda.T @ np.kron(J_n, I_T) @ u_tilda) / (u_tilda.T @ u_tilda))) ** 2
        return LM_C_time
    else:
        raise ValueError(f"'how' parameter should be either 'Entity' or 'Time' and not {how}.")
    
def marg_LM_stat(restricted_model: RandomEffectsResults, how: str): 
    if how.lower() == 'time':    
        T = restricted_model.time_info.total.astype('int')
        n = restricted_model.entity_info.total.astype('int')
    elif how.lower() == 'entity':
        n = restricted_model.time_info.total.astype('int')
        T = restricted_model.entity_info.total.astype('int')
    else:
        raise ValueError(f"'how' parameter should be either 'Entity' or 'Time' and not {how}.")

    J_n = np.ones(shape = (n, n))
    J_T = np.ones(shape = (T, T))
    
    J_n_bar = (1 / n) * J_n
    J_T_bar = (1 / T) * J_T

    I_n = np.identity(n = n)
    I_T = np.identity(n = T)

    E_n = I_n - J_n_bar
    E_T = I_T - J_T_bar

    u_tilda = restricted_model.resids

    if how.lower() == 'entity':
        sigma_v_sq = (1 / T*(n - 1)) * u_tilda.T @ np.kron(E_n, I_T) @ u_tilda
        sigma_2_sq = (1 / T) * u_tilda.T @ np.kron(J_n_bar, I_T) @ u_tilda

        Q_1 = (1 / sigma_2_sq) * u_tilda.T @ np.kron(J_n_bar, J_T_bar) @ u_tilda
        Q_2 = (1 / (n - 1)*sigma_v_sq) * u_tilda.T @ np.kron(E_n, J_T_bar) @ u_tilda

        LM_M_entity = ((np.sqrt(T) * sigma_2_sq * sigma_v_sq) / np.sqrt(2 * (T - 1) * (sigma_v_sq ** 2 + (n - 1) * sigma_2_sq ** 2))) *\
             ((1/sigma_2_sq) * (Q_1 - 1) + ((n-1) / sigma_v_sq) * (Q_2 - 1))
        
        return LM_M_entity
    
    else:
        sigma_v_sq = (1 / T*(n - 1)) * u_tilda.T @ np.kron(I_n, E_T) @ u_tilda
        sigma_1_sq = (1 / n) * u_tilda.T @ np.kron(I_n, J_T_bar) @ u_tilda

        R_1 = (1 / sigma_1_sq) * u_tilda.T @ np.kron(J_T_bar, J_n_bar) @ u_tilda
        R_2 = (1 / (T - 1)*sigma_v_sq) * u_tilda.T @ np.kron(J_n_bar, E_T) @ u_tilda

        LM_M_time = ((np.sqrt(n) * sigma_1_sq * sigma_v_sq) / np.sqrt(2 * (n - 1) * (sigma_v_sq ** 2 + (T - 1) * sigma_1_sq ** 2))) *\
             ((1/sigma_1_sq) * (R_1 - 1) + ((T-1) / sigma_v_sq) * (R_2 - 1))
        
        return LM_M_time

def joint_LM_stat(restricted_model: PanelResults): 
    return cond_LM_stat(restricted_model, 'Entity') + cond_LM_stat(restricted_model, 'Time')

#### Joint LM-Test

In [ ]:
pooled_ols_res = load_pickle('model/pooled_ols.pickle')

In [ ]:
joint_stat = joint_LM_stat(pooled_ols_res)

joint_p = 1 - stats.chi2.cdf(joint_stat, 1)

print(f"Joint LM Statistic for 2-way Random Effects: {joint_stat}")
print(f"p-value of Joint LM test for 2-way Random Effects: {joint_p}")


#### Conditional LM-Test for Entity Random Effect

In [ ]:
cond_entity_stat = cond_LM_stat(pooled_ols_res, how= 'entity')

cond_entity_p = 1 - stats.chi2.cdf(cond_entity_stat, 1)

print(f"Conditional LM Statistic for Entity Random Effects: {cond_entity_stat}")
print(f"p-value of Conditional LM test for Entity Random ffects: {cond_entity_p}")


#### Conditional LM-Test for Time Random Effect

In [ ]:
cond_time_stat = cond_LM_stat(pooled_ols_res, how= 'time')

cond_time_p = 1 - stats.chi2.cdf(cond_time_stat, 1)

print(f"Conditional LM Statistic for Time Random Effects: {cond_time_stat}")
print(f"p-value of Conditional LM test for Time Random Effects: {cond_time_p}")

#### Marginal LM-Test for Entity Random Effect

In [ ]:
marg_entity_stat = marg_LM_stat(time_re_fit, how= 'entity')

marg_entity_p = 1 - stats.chi2.cdf(marg_entity_stat, 1)

print(f"Marginal LM Statistic for Entity Random Effects: {marg_entity_stat}")
print(f"p-value of Marginal LM test for Entity Random ffects: {marg_entity_p}")

In [ ]:
marg_time_stat = marg_LM_stat(entity_re_fit, how= 'time')

marg_time_p = stats.chi2.cdf(marg_time_stat, 1)

print(f"Marginal LM Statistic for Time Random Effects: {marg_time_stat}")
print(f"p-value of Marginal LM test for Time Random ffects: {marg_time_p}")

From the results of the LM-test, only the joint LM and conditional LM test for entity effects were significant. However, the significance of the joint LM test is powered by the significance of the conditional LM test for entity as can be seen from the insignificance of the marginal LM tests. Thus, we should strongly consider the 1-way entity random effects model over the 2-way random effects model.

#### Log-Likelihood Ratio Test

In [ ]:
def lr_test(restricted_model: PanelResults|RandomEffectsResults|PanelEffectsResults, unrestricted_model: PanelResults|RandomEffectsResults|PanelEffectsResults, df: int= 1):
    try:
        res_loglik = restricted_model.loglik
    except:
        res_loglik = restricted_model.llf

    try:
        unres_loglik = unrestricted_model.loglik
    except:
        unres_loglik = unrestricted_model.llf


    lr_stat = -2 * (res_loglik - unres_loglik)
        
    lr_p = 1 - stats.chi2.cdf(lr_stat, df)

    print(f"Log-Likelihood Test Statistic: {lr_stat}")
    print(f"Log-Likelihood Test Statistic: {lr_p}")


#### Conditional Entity LR Test

In [ ]:
lr_test(pooled_ols_res, entity_re_fit, 1)

#### Conditional Time LR Test

In [ ]:
lr_test(pooled_ols_res, time_re_fit, 1)

#### Joint LR Test

In [ ]:
lr_test(pooled_ols_res, tw_re_fit, 2)

#### Conditional Time LR Test

In [ ]:
lr_test(entity_re_fit, tw_re_fit, 1)

#### Conditional Time LR Test

In [ ]:
lr_test(time_re_fit, tw_re_fit, 1)

### Correlated Random Effects Model

#### Correlated Entity Random Effects Model

In [ ]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = sgx[endo_var]

mean_exog_vars = ['avg' + var for var in exog_vars]
sgx[mean_exog_vars] = sgx[exog_vars].groupby(level= 'Company Code').transform('mean')

exog = sm.add_constant(sgx[exog_vars + mean_exog_vars])

entity_cre_mod = RandomEffects(endo, exog)

entity_cre_fit = entity_cre_mod.fit()

print(entity_cre_fit.summary)

#### Correlated 2-Way Random Effects Model

In [ ]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

endo = sgx[endo_var]

mean_exog_vars = ['avg' + var for var in exog_vars]
sgx[mean_exog_vars] = sgx[exog_vars].groupby(level= 'Company Code').transform('mean')

exog = sm.add_constant(sgx[exog_vars + mean_exog_vars])

entity_panel, time_panel = sgx['Company Code'], sgx['Year']

tw_cre_mod = TwoWayRandomEffects(endo, exog, entity_panel, time_panel)

tw_cre_fit = tw_cre_mod.fit()

print(tw_cre_fit.summary())

## Hausman Test

In [ ]:
def hausman_test(fixed_effects: PanelEffectsResults, random_effects: RandomEffectsResults|RegressionResults):    
    # (I) find overlapping coefficients:
    common_coef = list(set(fixed_effects.params.index).intersection(random_effects.params.index))

    # (II) calculate differences between FE and RE:
    b_diff = np.array(fixed_effects.params[common_coef] - random_effects.params[common_coef])
    df = len(b_diff)
    b_diff.reshape((df, 1))
    
    b_fe_cov = fixed_effects.cov
    try:
        b_re_cov = random_effects.cov
    except:
        b_re_cov = random_effects.cov_params()

    b_cov_diff = np.array(b_fe_cov.loc[common_coef, common_coef] -
                        b_re_cov.loc[common_coef, common_coef])
    b_cov_diff.reshape((df, df))

    # (III) calculate test statistic:
    hausman_stat = abs(np.transpose(b_diff) @ np.linalg.inv(b_cov_diff) @ b_diff)
    hausman_p = 1 - stats.chi2.cdf(hausman_stat, df)

    print(f"Hausman Test Statistic: {hausman_stat}")
    print(f"Hausman Test Statistic: {hausman_p}")

In [ ]:
tw_fe_fit = load_pickle('model/two_way_fe.pickle')

entity_fe_fit = load_pickle('model/entity_fe.pickle')

### 2-way FE vs. 2-way RE

In [ ]:
hausman_test(tw_fe_fit, tw_re_fit)

### Entity FE vs. Entity RE

In [ ]:
hausman_test(entity_fe_fit, entity_re_fit)

From the Hausman tests conducted, the p-values of all tests are significant at all reasonable levels of significance. Thus, a fixed effects model is preferred over the random effects model. 

### Time FE vs. Time RE

### CRE vs. RE Test

#### Entity CRE vs. Entity RE

In [ ]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

mean_exog_vars = ['avg' + var for var in exog_vars]

hypothesis = " = ".join(exog_vars + mean_exog_vars) + " = 0"
wald_test = entity_cre_fit.wald_test(formula= hypothesis)

print(wald_test)

#### 2-Way CRE vs. 2-Way RE

In [ ]:
endo_var = 'LEVERAGE'
exog_vars = ['SIZE',
             'PROFITABILITY',
             'TANG',
             'LIQUID',
             'MCAP',
             'SOLV']

mean_exog_vars = ['avg' + var for var in exog_vars]

hypothesis_matrix = " = ".join(mean_exog_vars) + ' = 0'
wald_test = tw_cre_fit.wald_test(hypothesis_matrix, use_f= False)

print(f"Wald-Test Statistic for 2-Way Correlated Random Effects: {wald_test.df_denom}")
print(f"Wald-Test Statistic for 2-Way Correlated Random Effects: {wald_test.statistic[0][0]}")
print(f"Wald-Test Statistic for 2-Way Correlated Random Effects: {wald_test.pvalue.item()}")

From the Wald-Tests conducted between the random effects and correlated random effects models, we have to reject the linearity constraint hypothesis and conclude that the CRE model is preferred. This is expected as we have already tested and accepted the significance of the within effects in our model.

### LM Test for 2-Way CRE

In [ ]:
marg_cre_entity_stat = marg_LM_stat(restricted_model= entity_cre_fit, how= 'Entity')
marg_cre_entity_p = 1 - stats.chi2.cdf(marg_cre_entity_stat, 1)

print(f"Marginal LM Statistic for Entity Correlated Random Effects: {marg_cre_entity_stat}")
print(f"p-value of Marginal LM test for Entity Correlated Random ffects: {marg_cre_entity_p}")

From the LM test conducted between the 2-way and entity correlated random effects, the p-value of 1.0 suggests that a 2-way model should not be preferred.

### Cluster Robust Entity CRE

## Conclusion

In [ ]:
pooled_ols_res.time_info